In [1]:
from utils import intake_data

## Importing and Scale Data

Removed 3 features: days, ventilator weaning, ventilator free days

In [2]:
data_pre = intake_data()
data_x = data_pre.drop(["death","days.1","ventilator weaning = 1", "VFD ","days"], axis=1)
data_y = data_pre["death"]

Scaled using MinMaxScaler() because that's what we used in assignment 2

In [5]:
x_mm = data_x 
y_mm = data_y 
from sklearn.preprocessing import MinMaxScaler 
scale = MinMaxScaler()
scale.fit(x_mm)

MinMaxScaler(copy=True, feature_range=(0, 1))

## Finding best parameters

Finding best number of n neighbors:

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_list = []
for i in range(1,31): 
    param_list.append(i)
parameters = {'n_neighbors':param_list}
gs = GridSearchCV(KNeighborsClassifier(), parameters, cv = 10)
gs_results = gs.fit(x_mm,y_mm)
gs.best_params_

{'n_neighbors': 2}

Running cross validation loop with n = 2

In [14]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

my_model = KNeighborsClassifier(n_neighbors = 2)
my_model.fit(x_mm, y_mm)
cross_val_score(my_model, x_mm, y_mm, cv = 10)
y_predict = cross_val_predict(my_model, x_mm, y_mm, cv=10)
print(classification_report(y_mm, y_predict))

              precision    recall  f1-score   support

         0.0       0.67      0.95      0.79       128
         1.0       0.59      0.14      0.23        69

    accuracy                           0.66       197
   macro avg       0.63      0.55      0.51       197
weighted avg       0.64      0.66      0.59       197



In [17]:
cross_val_score(gs, x_mm, y_mm, cv=10)
y_predict = cross_val_predict(gs, x_mm, y_mm, cv=10)
print(classification_report(y_mm, y_predict))

              precision    recall  f1-score   support

         0.0       0.65      0.96      0.78       128
         1.0       0.44      0.06      0.10        69

    accuracy                           0.64       197
   macro avg       0.55      0.51      0.44       197
weighted avg       0.58      0.64      0.54       197



Performing PCA dimensionality reduction

In [22]:
# Define a pipeline to search for the best combination of 
# PCA dimensions and n_neighbors.

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

#crate a scaler
scaler = MinMaxScaler()

#create a PCA
pca = PCA()

#create a KNN classifier
knn = KNeighborsClassifier()

#create a pipeline that does scaling, then PCA, then KNN
pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('knn', knn)])

#Set up the parameters you want to tune for each of your pipeline steps
#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': list(range(1, 19)), #find how many principal componenet to keep
    'knn__n_neighbors': list(range(1, 30)),  #find the best value of k
}

# pass the pipeline and the parameters into a GridSearchCV with a 5-fold cross validation
gs = GridSearchCV(pipe, param_grid, cv=5)
# call fit() on the GridSearchCV and pass in the unscaled data (X_values, Y_values)
x = data_x
y = data_y
gs.fit(x,y)
# print out the best_score_ and best_params_ from the GridSearchCV
print(gs.best_score_)
print(gs.best_params_)

0.7111538461538461
{'knn__n_neighbors': 9, 'pca__n_components': 6}


## Finding final accuracy score

In [23]:
from sklearn.metrics import accuracy_score
cross_val_score(gs, x, y, cv=5)
y_predict = cross_val_predict(gs, x, y, cv=5)
accuracy_score(y, y_predict)

0.6598984771573604